In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [2]:
# Define updated classes and functions
class Time_Series_Dataset(Dataset):
    def __init__(self, inputs, decoder_inputs, outputs):
        self.inputs = inputs
        self.decoder_inputs = decoder_inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x = self.inputs[idx]
        decoder_input = self.decoder_inputs[idx]
        y = self.outputs[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(decoder_input, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        out, (hidden, cell) = self.lstm(x, (hidden, cell))
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        out, (hidden, cell) = self.lstm(x, (hidden, cell))
        out = self.fc(out)
        return out, hidden, cell

class EncoderDecoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(EncoderDecoderLSTM, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, num_layers)
        self.decoder = Decoder(1, hidden_size, output_size, num_layers)

    def forward(self, encoder_inputs, decoder_inputs):
        hidden, cell = self.encoder(encoder_inputs)
        decoder_inputs = decoder_inputs.unsqueeze(-1)  # Add feature dimension to decoder inputs
        outputs, hidden, cell = self.decoder(decoder_inputs, hidden, cell)
        return outputs

def split_data(features, target, input_steps, output_steps, train_ratio, seed):
    X, y, decoder_inputs = [], [], []
    total_size = input_steps + output_steps
    for i in range(len(features) - total_size + 1):
        X.append(features[i:i + input_steps])
        y.append(target[i + input_steps:i + total_size])
        decoder_inputs.append(target[i + input_steps - 1:i + input_steps + output_steps - 1])
    
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = train_test_split(
        X, y, decoder_inputs, train_size=train_ratio, random_state=seed
    )
    return X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test
    
pm = "\u00B1"

In [3]:
def lorenz(xyz, *, s=10, r=28, b=2.667):
    x, y, z = xyz
    x_dot = s*(y - x)
    y_dot = r*x - y - x*z
    z_dot = x*y - b*z
    return np.array([x_dot, y_dot, z_dot])

dt = 0.01
num_steps = 10000

xyzs = np.empty((num_steps + 1, 3))  # Need one more for the initial values
xyzs[0] = (0., 1., 1.05)  # Set initial values
# Step through "time", calculating the partial derivatives at the current point
# and using them to estimate the next point
for i in range(num_steps):
    xyzs[i + 1] = xyzs[i] + lorenz(xyzs[i]) * dt

# break it down to 3 univariate time series
x_dimension = xyzs[:, 0]
y_dimension = xyzs[:, 1]
z_dimension = xyzs[:, 2]

x_dimension = x_dimension.flatten()
x_dimension = pd.Series(x_dimension, name='Value')
y_dimension = y_dimension.flatten()
y_dimension = pd.Series(y_dimension, name='Value')
z_dimension = z_dimension.flatten()
z_dimension = pd.Series(z_dimension, name='Value')

x_dimension.index = range(len(x_dimension))
y_dimension.index = range(len(y_dimension))
z_dimension.index = range(len(z_dimension))
# print(x_dimension), print(y_dimension), print(z_dimension)

scaler = MinMaxScaler(feature_range=(0, 1))
x_reshaped = np.array(x_dimension).reshape(-1, 1)
y_reshaped = np.array(y_dimension).reshape(-1, 1)
z_reshaped = np.array(z_dimension).reshape(-1, 1)
x_scaled = scaler.fit_transform(x_reshaped).flatten()
y_scaled = scaler.fit_transform(y_reshaped).flatten()
z_scaled = scaler.fit_transform(z_reshaped).flatten()

In [4]:
input_steps = 5
output_steps = 10
train_ratio = 0.8
seed = 5925
num_experiments = 30   # default: 30

rmse, mae, mape = [], [], []
rmse_steps = [[] for _ in range(output_steps)]
mae_steps = [[] for _ in range(output_steps)]
mape_steps = [[] for _ in range(output_steps)]

train_rmse, train_mae, train_mape = [], [], []
train_rmse_steps = [[] for _ in range(output_steps)]
train_mae_steps = [[] for _ in range(output_steps)]
train_mape_steps = [[] for _ in range(output_steps)]

In [5]:
# X Dimension
features = x_scaled
target = x_scaled
for exp in range(num_experiments):
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = split_data(
        features, target, input_steps, output_steps, train_ratio, seed
    )
    train_dataset = Time_Series_Dataset(X_train, decoder_inputs_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, decoder_inputs_test, y_test)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    # Training loop
    input_size = 1  # number of features
    hidden_size = 100 # Default: 100
    output_size = 1  # predicting 1 value per time step
    num_layers = 2  # Two LSTM networks with a time-distributed layer
    
    model = EncoderDecoderLSTM(input_size, hidden_size, output_size, num_layers)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    num_epochs = 100 # default: 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate the model on the test set
    model.eval()
    y_pred = []
    y_test = []
    
    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in test_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            
            y_pred.append(outputs.numpy())
            y_test.append(targets.numpy())
    
    # Convert lists to numpy arrays and remove last dimension
    y_pred = np.concatenate(y_pred, axis=0).squeeze(-1)
    y_test = np.concatenate(y_test, axis=0).squeeze(-1)

    mse = mean_squared_error(y_test, y_pred)
    rmse.append(np.sqrt(mse))
    
    for step in range(output_steps):
        mse_step = mean_squared_error(y_pred[:, step], y_test[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))

    # Inverse Transform
    scaler = MinMaxScaler()
    scaler.fit(target.reshape(-1, 1))
    predicted_values = scaler.inverse_transform(y_pred)
    actual_values = scaler.inverse_transform(y_test)
    
    mae.append(mean_absolute_error(actual_values, predicted_values))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_values))
    
    actual_values_steps = list(zip(*actual_values))
    predicted_values_steps = list(zip(*predicted_values))
    
    for step in range(output_steps):
        mae_steps[step].append(mean_absolute_error(actual_values_steps[step], predicted_values_steps[step]))
        mape_steps[step].append(mean_absolute_percentage_error(actual_values_steps[step], predicted_values_steps[step]))

    # Evaluate the model on the train set
    y_train_pred = []
    y_train_actual = []

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            
            y_train_pred.append(outputs.numpy())
            y_train_actual.append(targets.numpy())

    # Convert lists to numpy arrays and remove last dimension
    y_train_pred = np.concatenate(y_train_pred, axis=0).squeeze(-1)
    y_train_actual = np.concatenate(y_train_actual, axis=0).squeeze(-1)

    train_mse = mean_squared_error(y_train_actual, y_train_pred)
    train_rmse.append(np.sqrt(train_mse))
    
    for step in range(output_steps):
        train_mse_step = mean_squared_error(y_train_pred[:, step], y_train_actual[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))

    # Inverse Transform
    train_predicted_values = scaler.inverse_transform(y_train_pred)
    train_actual_values = scaler.inverse_transform(y_train_actual)
    
    train_mae.append(mean_absolute_error(train_actual_values, train_predicted_values))
    train_mape.append(mean_absolute_percentage_error(train_actual_values, train_predicted_values))
    
    train_actual_values_steps = list(zip(*train_actual_values))
    train_predicted_values_steps = list(zip(*train_predicted_values))
    
    for step in range(output_steps):
        train_mae_steps[step].append(mean_absolute_error(train_actual_values_steps[step], train_predicted_values_steps[step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(train_actual_values_steps[step], train_predicted_values_steps[step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1


Experiment 1/30 done
Experiment 2/30 done
Experiment 3/30 done
Experiment 4/30 done
Experiment 5/30 done
Experiment 6/30 done
Experiment 7/30 done
Experiment 8/30 done
Experiment 9/30 done
Experiment 10/30 done
Experiment 11/30 done
Experiment 12/30 done
Experiment 13/30 done
Experiment 14/30 done
Experiment 15/30 done
Experiment 16/30 done
Experiment 17/30 done
Experiment 18/30 done
Experiment 19/30 done
Experiment 20/30 done
Experiment 21/30 done
Experiment 22/30 done
Experiment 23/30 done
Experiment 24/30 done
Experiment 25/30 done
Experiment 26/30 done
Experiment 27/30 done
Experiment 28/30 done
Experiment 29/30 done
Experiment 30/30 done


In [6]:
print(f"Lorenz X ED-LSTM Regression: After {num_experiments} experimental runs, here are the results:")

# Test dataset results
print(f"Across {output_steps} predictive time steps on the test dataset, " +
      f"Avg RMSE: {np.mean(rmse):.4f} {pm} {np.std(rmse):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} {pm} {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% {pm} {np.std(mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the test dataset, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} {pm} {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% {pm} {np.std(mape_steps[step]) * 100:.3f}%"
    )

# Train dataset results
print(f"Across {output_steps} predictive time steps on the train dataset, " +
      f"Avg RMSE: {np.mean(train_rmse):.4f} {pm} {np.std(train_rmse):.4f}, " +
      f"Avg MAE: {np.mean(train_mae):.2f} {pm} {np.std(train_mae):.2f}, " +
      f"Avg MAPE: {np.mean(train_mape)*100:.3f}% {pm} {np.std(train_mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the train dataset, "
        f"Avg RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(train_mae_steps[step]):.2f} {pm} {np.std(train_mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(train_mape_steps[step]) * 100:.3f}% {pm} {np.std(train_mape_steps[step]) * 100:.3f}%")


Lorenz X ED-LSTM Regression: After 30 experimental runs, here are the results:
Across 10 predictive time steps on the test dataset, Avg RMSE: 0.0015 ± 0.0003, Avg MAE: 0.00 ± 0.00, Avg MAPE: 96189305600.000% ± 84621241600.000%
At time step 1 on the test dataset, Avg RMSE: 0.0017 ± 0.0004, Avg MAE: 0.00 ± 0.00, Avg MAPE: 356900838400.000% ± 606089984000.000%
At time step 2 on the test dataset, Avg RMSE: 0.0021 ± 0.0006, Avg MAE: 0.00 ± 0.00, Avg MAPE: 86285491200.000% ± 199022425600.000%
At time step 3 on the test dataset, Avg RMSE: 0.0021 ± 0.0006, Avg MAE: 0.00 ± 0.00, Avg MAPE: 31821292800.000% ± 124547968000.000%
At time step 4 on the test dataset, Avg RMSE: 0.0019 ± 0.0006, Avg MAE: 0.00 ± 0.00, Avg MAPE: 39906064000.000% ± 104057043200.000%
At time step 5 on the test dataset, Avg RMSE: 0.0015 ± 0.0006, Avg MAE: 0.00 ± 0.00, Avg MAPE: 58613952000.000% ± 142616115200.000%
At time step 6 on the test dataset, Avg RMSE: 0.0012 ± 0.0004, Avg MAE: 0.00 ± 0.00, Avg MAPE: 47348134400.000% 

In [7]:
pd.DataFrame(actual_values).to_csv('lorenzx_uni_edlstm_classic_actual.csv')
pd.DataFrame(predicted_values).to_csv('lorenzx_uni_edlstm_classic_pred.csv')
pd.DataFrame(rmse_steps).transpose().to_csv('lorenzx_uni_edlstm_classic_rmse.csv')
pd.DataFrame(mae_steps).transpose().to_csv('lorenzx_uni_edlstm_classic_mae.csv')
pd.DataFrame(mape_steps).transpose().to_csv('lorenzx_uni_edlstm_classic_mape.csv')

In [8]:
## Y-Dimension
input_steps = 5
output_steps = 10
train_ratio = 0.8
seed = 5925
num_experiments = 30   # default: 30

rmse, mae, mape = [], [], []
rmse_steps = [[] for _ in range(output_steps)]
mae_steps = [[] for _ in range(output_steps)]
mape_steps = [[] for _ in range(output_steps)]

train_rmse, train_mae, train_mape = [], [], []
train_rmse_steps = [[] for _ in range(output_steps)]
train_mae_steps = [[] for _ in range(output_steps)]
train_mape_steps = [[] for _ in range(output_steps)]

In [9]:
# Y Dimension
features = y_scaled
target = y_scaled
for exp in range(num_experiments):
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = split_data(
        features, target, input_steps, output_steps, train_ratio, seed
    )
    train_dataset = Time_Series_Dataset(X_train, decoder_inputs_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, decoder_inputs_test, y_test)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    # Training loop
    input_size = 1  # number of features
    hidden_size = 100 # Default: 100
    output_size = 1  # predicting 1 value per time step
    num_layers = 2  # Two LSTM networks with a time-distributed layer
    
    model = EncoderDecoderLSTM(input_size, hidden_size, output_size, num_layers)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    num_epochs = 100 # default: 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate the model on the test set
    model.eval()
    y_pred = []
    y_test = []
    
    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in test_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            
            y_pred.append(outputs.numpy())
            y_test.append(targets.numpy())
    
    # Convert lists to numpy arrays and remove last dimension
    y_pred = np.concatenate(y_pred, axis=0).squeeze(-1)
    y_test = np.concatenate(y_test, axis=0).squeeze(-1)

    mse = mean_squared_error(y_test, y_pred)
    rmse.append(np.sqrt(mse))
    
    for step in range(output_steps):
        mse_step = mean_squared_error(y_pred[:, step], y_test[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))

    # Inverse Transform
    scaler = MinMaxScaler()
    scaler.fit(target.reshape(-1, 1))
    predicted_values = scaler.inverse_transform(y_pred)
    actual_values = scaler.inverse_transform(y_test)
    
    mae.append(mean_absolute_error(actual_values, predicted_values))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_values))
    
    actual_values_steps = list(zip(*actual_values))
    predicted_values_steps = list(zip(*predicted_values))
    
    for step in range(output_steps):
        mae_steps[step].append(mean_absolute_error(actual_values_steps[step], predicted_values_steps[step]))
        mape_steps[step].append(mean_absolute_percentage_error(actual_values_steps[step], predicted_values_steps[step]))

    # Evaluate the model on the train set
    y_train_pred = []
    y_train_actual = []

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            
            y_train_pred.append(outputs.numpy())
            y_train_actual.append(targets.numpy())

    # Convert lists to numpy arrays and remove last dimension
    y_train_pred = np.concatenate(y_train_pred, axis=0).squeeze(-1)
    y_train_actual = np.concatenate(y_train_actual, axis=0).squeeze(-1)

    train_mse = mean_squared_error(y_train_actual, y_train_pred)
    train_rmse.append(np.sqrt(train_mse))
    
    for step in range(output_steps):
        train_mse_step = mean_squared_error(y_train_pred[:, step], y_train_actual[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))

    # Inverse Transform
    train_predicted_values = scaler.inverse_transform(y_train_pred)
    train_actual_values = scaler.inverse_transform(y_train_actual)
    
    train_mae.append(mean_absolute_error(train_actual_values, train_predicted_values))
    train_mape.append(mean_absolute_percentage_error(train_actual_values, train_predicted_values))
    
    train_actual_values_steps = list(zip(*train_actual_values))
    train_predicted_values_steps = list(zip(*train_predicted_values))
    
    for step in range(output_steps):
        train_mae_steps[step].append(mean_absolute_error(train_actual_values_steps[step], train_predicted_values_steps[step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(train_actual_values_steps[step], train_predicted_values_steps[step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1


Experiment 1/30 done
Experiment 2/30 done
Experiment 3/30 done
Experiment 4/30 done
Experiment 5/30 done
Experiment 6/30 done
Experiment 7/30 done
Experiment 8/30 done
Experiment 9/30 done
Experiment 10/30 done
Experiment 11/30 done
Experiment 12/30 done
Experiment 13/30 done
Experiment 14/30 done
Experiment 15/30 done
Experiment 16/30 done
Experiment 17/30 done
Experiment 18/30 done
Experiment 19/30 done
Experiment 20/30 done
Experiment 21/30 done
Experiment 22/30 done
Experiment 23/30 done
Experiment 24/30 done
Experiment 25/30 done
Experiment 26/30 done
Experiment 27/30 done
Experiment 28/30 done
Experiment 29/30 done
Experiment 30/30 done


In [10]:
print(f"Lorenz Y ED-LSTM Regression: After {num_experiments} experimental runs, here are the results:")

# Test dataset results
print(f"Across {output_steps} predictive time steps on the test dataset, " +
      f"Avg RMSE: {np.mean(rmse):.4f} {pm} {np.std(rmse):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} {pm} {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% {pm} {np.std(mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the test dataset, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} {pm} {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% {pm} {np.std(mape_steps[step]) * 100:.3f}%"
    )

# Train dataset results
print(f"Across {output_steps} predictive time steps on the train dataset, " +
      f"Avg RMSE: {np.mean(train_rmse):.4f} {pm} {np.std(train_rmse):.4f}, " +
      f"Avg MAE: {np.mean(train_mae):.2f} {pm} {np.std(train_mae):.2f}, " +
      f"Avg MAPE: {np.mean(train_mape)*100:.3f}% {pm} {np.std(train_mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the train dataset, "
        f"Avg RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(train_mae_steps[step]):.2f} {pm} {np.std(train_mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(train_mape_steps[step]) * 100:.3f}% {pm} {np.std(train_mape_steps[step]) * 100:.3f}%")


Lorenz Y ED-LSTM Regression: After 30 experimental runs, here are the results:
Across 10 predictive time steps on the test dataset, Avg RMSE: 0.0017 ± 0.0004, Avg MAE: 0.00 ± 0.00, Avg MAPE: 111640896000.000% ± 96261696000.000%
At time step 1 on the test dataset, Avg RMSE: 0.0024 ± 0.0007, Avg MAE: 0.00 ± 0.00, Avg MAPE: 148140684800.000% ± 449500057600.000%
At time step 2 on the test dataset, Avg RMSE: 0.0026 ± 0.0008, Avg MAE: 0.00 ± 0.00, Avg MAPE: 152552396800.000% ± 390675558400.000%
At time step 3 on the test dataset, Avg RMSE: 0.0021 ± 0.0007, Avg MAE: 0.00 ± 0.00, Avg MAPE: 126377036800.000% ± 270701132800.000%
At time step 4 on the test dataset, Avg RMSE: 0.0016 ± 0.0007, Avg MAE: 0.00 ± 0.00, Avg MAPE: 52290963200.000% ± 151874240000.000%
At time step 5 on the test dataset, Avg RMSE: 0.0012 ± 0.0004, Avg MAE: 0.00 ± 0.00, Avg MAPE: 163072652800.000% ± 361087411200.000%
At time step 6 on the test dataset, Avg RMSE: 0.0010 ± 0.0003, Avg MAE: 0.00 ± 0.00, Avg MAPE: 70234592000.0

In [11]:
pd.DataFrame(actual_values).to_csv('lorenzy_uni_edlstm_classic_actual.csv')
pd.DataFrame(predicted_values).to_csv('lorenzy_uni_edlstm_classic_pred.csv')
pd.DataFrame(rmse_steps).transpose().to_csv('lorenzy_uni_edlstm_classic_rmse.csv')
pd.DataFrame(mae_steps).transpose().to_csv('lorenzy_uni_edlstm_classic_mae.csv')
pd.DataFrame(mape_steps).transpose().to_csv('lorenzy_uni_edlstm_classic_mape.csv')

In [12]:
## Z-Dimension
input_steps = 5
output_steps = 10
train_ratio = 0.8
seed = 5925
num_experiments = 30   # default: 30

rmse, mae, mape = [], [], []
rmse_steps = [[] for _ in range(output_steps)]
mae_steps = [[] for _ in range(output_steps)]
mape_steps = [[] for _ in range(output_steps)]

train_rmse, train_mae, train_mape = [], [], []
train_rmse_steps = [[] for _ in range(output_steps)]
train_mae_steps = [[] for _ in range(output_steps)]
train_mape_steps = [[] for _ in range(output_steps)]

In [13]:
# Z Dimension
features = z_scaled
target = z_scaled
for exp in range(num_experiments):
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = split_data(
        features, target, input_steps, output_steps, train_ratio, seed
    )
    train_dataset = Time_Series_Dataset(X_train, decoder_inputs_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, decoder_inputs_test, y_test)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    # Training loop
    input_size = 1  # number of features
    hidden_size = 100 # Default: 100
    output_size = 1  # predicting 1 value per time step
    num_layers = 2  # Two LSTM networks with a time-distributed layer
    
    model = EncoderDecoderLSTM(input_size, hidden_size, output_size, num_layers)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    num_epochs = 100 # default: 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate the model on the test set
    model.eval()
    y_pred = []
    y_test = []
    
    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in test_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            
            y_pred.append(outputs.numpy())
            y_test.append(targets.numpy())
    
    # Convert lists to numpy arrays and remove last dimension
    y_pred = np.concatenate(y_pred, axis=0).squeeze(-1)
    y_test = np.concatenate(y_test, axis=0).squeeze(-1)

    mse = mean_squared_error(y_test, y_pred)
    rmse.append(np.sqrt(mse))
    
    for step in range(output_steps):
        mse_step = mean_squared_error(y_pred[:, step], y_test[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))

    # Inverse Transform
    scaler = MinMaxScaler()
    scaler.fit(target.reshape(-1, 1))
    predicted_values = scaler.inverse_transform(y_pred)
    actual_values = scaler.inverse_transform(y_test)
    
    mae.append(mean_absolute_error(actual_values, predicted_values))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_values))
    
    actual_values_steps = list(zip(*actual_values))
    predicted_values_steps = list(zip(*predicted_values))
    
    for step in range(output_steps):
        mae_steps[step].append(mean_absolute_error(actual_values_steps[step], predicted_values_steps[step]))
        mape_steps[step].append(mean_absolute_percentage_error(actual_values_steps[step], predicted_values_steps[step]))

    # Evaluate the model on the train set
    y_train_pred = []
    y_train_actual = []

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, output_size)
            
            y_train_pred.append(outputs.numpy())
            y_train_actual.append(targets.numpy())

    # Convert lists to numpy arrays and remove last dimension
    y_train_pred = np.concatenate(y_train_pred, axis=0).squeeze(-1)
    y_train_actual = np.concatenate(y_train_actual, axis=0).squeeze(-1)

    train_mse = mean_squared_error(y_train_actual, y_train_pred)
    train_rmse.append(np.sqrt(train_mse))
    
    for step in range(output_steps):
        train_mse_step = mean_squared_error(y_train_pred[:, step], y_train_actual[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))

    # Inverse Transform
    train_predicted_values = scaler.inverse_transform(y_train_pred)
    train_actual_values = scaler.inverse_transform(y_train_actual)
    
    train_mae.append(mean_absolute_error(train_actual_values, train_predicted_values))
    train_mape.append(mean_absolute_percentage_error(train_actual_values, train_predicted_values))
    
    train_actual_values_steps = list(zip(*train_actual_values))
    train_predicted_values_steps = list(zip(*train_predicted_values))
    
    for step in range(output_steps):
        train_mae_steps[step].append(mean_absolute_error(train_actual_values_steps[step], train_predicted_values_steps[step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(train_actual_values_steps[step], train_predicted_values_steps[step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1


Experiment 1/30 done
Experiment 2/30 done
Experiment 3/30 done
Experiment 4/30 done
Experiment 5/30 done
Experiment 6/30 done
Experiment 7/30 done
Experiment 8/30 done
Experiment 9/30 done
Experiment 10/30 done
Experiment 11/30 done
Experiment 12/30 done
Experiment 13/30 done
Experiment 14/30 done
Experiment 15/30 done
Experiment 16/30 done
Experiment 17/30 done
Experiment 18/30 done
Experiment 19/30 done
Experiment 20/30 done
Experiment 21/30 done
Experiment 22/30 done
Experiment 23/30 done
Experiment 24/30 done
Experiment 25/30 done
Experiment 26/30 done
Experiment 27/30 done
Experiment 28/30 done
Experiment 29/30 done
Experiment 30/30 done


In [14]:
print(f"Lorenz Z ED-LSTM Regression: After {num_experiments} experimental runs, here are the results:")

# Test dataset results
print(f"Across {output_steps} predictive time steps on the test dataset, " +
      f"Avg RMSE: {np.mean(rmse):.4f} {pm} {np.std(rmse):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} {pm} {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% {pm} {np.std(mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the test dataset, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} {pm} {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% {pm} {np.std(mape_steps[step]) * 100:.3f}%"
    )

# Train dataset results
print(f"Across {output_steps} predictive time steps on the train dataset, " +
      f"Avg RMSE: {np.mean(train_rmse):.4f} {pm} {np.std(train_rmse):.4f}, " +
      f"Avg MAE: {np.mean(train_mae):.2f} {pm} {np.std(train_mae):.2f}, " +
      f"Avg MAPE: {np.mean(train_mape)*100:.3f}% {pm} {np.std(train_mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the train dataset, "
        f"Avg RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(train_mae_steps[step]):.2f} {pm} {np.std(train_mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(train_mape_steps[step]) * 100:.3f}% {pm} {np.std(train_mape_steps[step]) * 100:.3f}%")


Lorenz Z ED-LSTM Regression: After 30 experimental runs, here are the results:
Across 10 predictive time steps on the test dataset, Avg RMSE: 0.0014 ± 0.0005, Avg MAE: 0.00 ± 0.00, Avg MAPE: 52638947200.000% ± 53385904000.000%
At time step 1 on the test dataset, Avg RMSE: 0.0015 ± 0.0004, Avg MAE: 0.00 ± 0.00, Avg MAPE: 53378659200.000% ± 140206976000.000%
At time step 2 on the test dataset, Avg RMSE: 0.0019 ± 0.0008, Avg MAE: 0.00 ± 0.00, Avg MAPE: 63918598400.000% ± 156357427200.000%
At time step 3 on the test dataset, Avg RMSE: 0.0020 ± 0.0009, Avg MAE: 0.00 ± 0.00, Avg MAPE: 82943571200.000% ± 139643033600.000%
At time step 4 on the test dataset, Avg RMSE: 0.0018 ± 0.0009, Avg MAE: 0.00 ± 0.00, Avg MAPE: 32912579200.000% ± 80179948800.000%
At time step 5 on the test dataset, Avg RMSE: 0.0014 ± 0.0008, Avg MAE: 0.00 ± 0.00, Avg MAPE: 42974022400.000% ± 125023398400.000%
At time step 6 on the test dataset, Avg RMSE: 0.0010 ± 0.0006, Avg MAE: 0.00 ± 0.00, Avg MAPE: 129281254400.000% ±

In [15]:
pd.DataFrame(actual_values).to_csv('lorenzz_uni_edlstm_classic_actual.csv')
pd.DataFrame(predicted_values).to_csv('lorenzz_uni_edlstm_classic_pred.csv')
pd.DataFrame(rmse_steps).transpose().to_csv('lorenzz_uni_edlstm_classic_rmse.csv')
pd.DataFrame(mae_steps).transpose().to_csv('lorenzz_uni_edlstm_classic_mae.csv')
pd.DataFrame(mape_steps).transpose().to_csv('lorenzz_uni_edlstm_classic_mape.csv')